In [1]:
from astropy.io import fits
import antropy as ant
import matplotlib.pyplot as plt
import numpy as np
# import dask.dataframe as dd
import pandas as pd
import os
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks, periodogram
from scipy.fft import fft
import tensorflow as tf
import tsfel
from scipy.signal import savgol_filter
from sklearn.model_selection import KFold
# from tsfresh import extract_relevant_features
# from tsfresh.feature_extraction import EfficientFCParameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split

In [2]:
positive_dips = []
negative_dips = []
def extract_features(time_series, sampling_rate=1.0, label=None):
    """
    Extracts a set of temporal and spectral features from a given time series.
    
    Parameters:
    - time_series: np.array, the input time series data
    - sampling_rate: float, the sampling rate of the time series (default is 1.0 Hz)
    
    Returns:
    - features: np.array, an array of extracted feature values
    """
    # Ensure the input is a numpy array
    time_series = np.array(time_series)

    # Temporal Features
    mean_value = np.mean(time_series)
    std_value = np.std(time_series)
    median_flux = np.median(time_series)
    min_flux = np.min(time_series)
    max_flux = np.max(time_series)
    skewness = skew(time_series)
    kurt = kurtosis(time_series)
    peak_to_peak = np.ptp(time_series)  # Peak-to-peak amplitude
    median_absolute_deviation = np.median(np.abs(time_series - np.median(time_series)))

    # Number of significant peaks
    peaks, _ = find_peaks(time_series)
    num_peaks = len(peaks)

    dips, _ = find_peaks(-time_series)
    num_dips = len(dips)
    if label == 1:
        positive_dips.append(num_dips)
    else:
        negative_dips.append(num_dips)
    if len(dips) > 1:
        duration_of_dips = np.mean(np.diff(dips))
    else:
        duration_of_dips = 0

    # Autocorrelation (lag-1)
    if len(time_series) > 1:
        autocorr = np.corrcoef(time_series[:-1], time_series[1:])[0, 1]
    else:
        autocorr = 0

    # Slope of the time series
    slope = (time_series[-1] - time_series[0]) / len(time_series)
    zero_crossings = np.where(np.diff(np.sign(time_series)))[0]
    zero_crossing_rate = len(zero_crossings) / len(time_series)

    # Spectral Features
    # Compute the Power Spectral Density (PSD) using periodogram
    freqs, power = periodogram(time_series, fs=sampling_rate)

    # Dominant frequency and its power
    if len(freqs) > 1:
        dominant_freq = freqs[np.argmax(power)]
        power_of_dominant_freq = np.max(power)
    else:
        dominant_freq = 0
        power_of_dominant_freq = 0

    # Combine all features into a single array
    features = np.array([
        mean_value, std_value, skewness, kurt, peak_to_peak,
        median_absolute_deviation, num_peaks, autocorr, slope,
        dominant_freq, power_of_dominant_freq, num_dips, median_flux, min_flux, max_flux, duration_of_dips
    ])

    return features


def extract_spectral_features(light_curve, sampling_rate=1.0):
    light_curve = np.array(light_curve)
    freqs, power = periodogram(light_curve, fs=sampling_rate)

    # Spectrogram Mean Coefficient
    spectrogram_mean = np.mean(power)

    # Fundamental Frequency and Max Power Spectrum
    if len(freqs) > 1:
        fundamental_frequency = freqs[np.argmax(power)]
        max_power_spectrum = np.max(power)
    else:
        fundamental_frequency = 0
        max_power_spectrum = 0

    # Spectral Centroid
    spectral_centroid = np.sum(freqs * power) / np.sum(power) if np.sum(power) > 0 else 0

    # Spectral Entropy
    normalized_power = power / np.sum(power) if np.sum(power) > 0 else power
    spectral_entropy = -np.sum(normalized_power * np.log2(normalized_power + 1e-10))

    # Spectral Kurtosis
    spectral_kurtosis = kurtosis(power)

    # Spectral Skewness
    spectral_skewness = skew(power)

    # Spectral Roll-off (85% energy)
    cumulative_energy = np.cumsum(power)
    spectral_roll_off = freqs[np.where(cumulative_energy >= 0.85 * cumulative_energy[-1])[0][0]] if len(
        cumulative_energy) > 0 else 0

    # Spectral Slope
    if len(freqs) > 1:
        spectral_slope = (power[-1] - power[0]) / (freqs[-1] - freqs[0])
    else:
        spectral_slope = 0

    # Wavelet Features
    wavelet = 'db1'  # Daubechies wavelet with one vanishing moment
    coeffs = pywt.wavedec(light_curve, wavelet, level=3)

    # Wavelet Energy
    wavelet_energy = np.sum([np.sum(c ** 2) for c in coeffs])

    # Wavelet Entropy
    wavelet_coeffs = np.concatenate(coeffs)
    normalized_wavelet_coeffs = wavelet_coeffs / np.sum(wavelet_coeffs) if np.sum(
        wavelet_coeffs) > 0 else wavelet_coeffs
    # Filter out zero or negative values before computing the log
    valid_coeffs = normalized_wavelet_coeffs[normalized_wavelet_coeffs > 0]
    wavelet_entropy = -np.sum(valid_coeffs * np.log2(valid_coeffs + 1e-10))

    # Combine all features into a single array
    features = np.array([
        spectrogram_mean, fundamental_frequency, max_power_spectrum, spectral_centroid,
        spectral_entropy, spectral_kurtosis, spectral_skewness, spectral_roll_off,
        spectral_slope, wavelet_energy, wavelet_entropy
    ])

    return features


def compute_additional_features(light_curve):
    features = []

    # Spectral Flatness
    power_spectrum = np.abs(np.fft.fft(light_curve)) ** 2
    #print("done 1")
    spectral_flatness = np.exp(np.mean(np.log(power_spectrum + 1e-10))) / np.mean(power_spectrum)
    #print("done 2")
    features.append(spectral_flatness)

    # Lagged Autocorrelations
    autocorr_lag_5 = np.corrcoef(light_curve[:-5], light_curve[5:])[0, 1]
    features.append(autocorr_lag_5)
    #print("done 3")

    # Zero-Crossing Rate
    zero_crossings = np.where(np.diff(np.sign(light_curve)))[0]
    zero_crossing_rate = len(zero_crossings) / len(light_curve)
    features.append(zero_crossing_rate)
    #print("done 4")

    # Approximate Entropy (using a simplified method)
    def approximate_entropy(U, m, r):
        N = len(U)

        def _phi(m):
            x = np.array([U[i: i + m] for i in range(N - m + 1)])
            C = np.sum(np.max(np.abs(x[:, None] - x[None, :]), axis=2) <= r, axis=0) / (N - m + 1)
            return np.sum(np.log(C)) / (N - m + 1)

        return abs(_phi(m + 1) - _phi(m))

    approx_entropy = ant.app_entropy(light_curve, order=2, metric='chebyshev')
    #approx_entropy = approximate_entropy(light_curve, m=2, r=0.2 * np.std(light_curve))
    #print("done 5")
    features.append(approx_entropy)

    return np.array(features)


def features_from_tsfel(light_curve, sampling_rate=1.0):
    lempel_ziv = tsfel.feature_extraction.features.lempel_ziv(light_curve)
    maximum_fractal_length = tsfel.feature_extraction.features.maximum_fractal_length(light_curve)
    return [lempel_ziv, maximum_fractal_length]


In [3]:
import sys

index = 1
positive_folder = "./data/positive/"
negative_folder = "./data/negative/"
# flux_array = []
# time_array = []
# id_list = []
labels = []
# label_obj = {}
extracted_features_list = []


def load_fits_data(folder, label):
    for filename in os.listdir(folder):
        if filename.endswith('.fits'):
            with fits.open(os.path.join(folder, filename)) as hdul:
                try:
                    global index
                    if index > 100:
                        break
                    data = hdul[1].data
                    header = hdul[1].header
                    #print(header)
                    time = data['TIME']
                    flux = data['PDCSAP_FLUX']
                    #print("flux", flux.shape)
                    obsmode = header.get('OBSMODE')
                    timedel = header.get('TIMEDEL') * 86400
                    sampling_rate = 1 / timedel
                    #print(obsmode, timedel, sampling_rate)

                    valid_indices = ~np.isnan(time) & ~np.isnan(flux)
                    time = time[valid_indices]
                    flux = flux[valid_indices]
                    # mean = np.mean(flux)
                    # std = np.std(flux)
                    # new_flux = (flux - mean) / std
                    smoothed_flux = savgol_filter(flux, window_length=201, polyorder=6)
                    detrended_flux = flux - smoothed_flux

                    #print("yoyoy")
                    # id_arr = np.array([index for i in range(flux.shape[0])])
                    # time_array.extend(time)
                    # flux_array.extend(flux)
                    # id_list.extend(id_arr)
                    # label_obj[index] = label
                    index += 1
                    # labels.append(label)
                    basic_features = extract_features(smoothed_flux, sampling_rate, label)
                    print("label, dips", label, basic_features[-5])
                    spectral_features = extract_spectral_features(smoothed_flux, sampling_rate)
                    #detrend_features = extract_features(detrended_flux, sampling_rate)
                    additional_features = compute_additional_features(smoothed_flux)
                    #from_tsfel = features_from_tsfel(smoothed_flux, sampling_rate)
                    #print(from_tsfel)
                    #print(basic_features)
                    X = np.concatenate((basic_features, spectral_features, additional_features))
                    # print(len(basic_features), len(spectral_features), len(additional_features))
                    #X.shape
                    #cfg = tsfel.get_features_by_domain('spectral')
                    #print(cfg)
                    # df = pd.DataFrame({
                    #     'time': time,
                    #     'flux': smoothed_flux
                    # })
                    #X = tsfel.time_series_features_extractor(cfg, df, fs=0.056)
                    extracted_features_list.append(X)
                    labels.append(label)
                    #print(X.shape)
                except ValueError as e:
                    print("ValueError error for filename", e)
                except AttributeError as e:
                    print("AttributeError error for filename", filename, e)
                except NameError as e:
                    print("NameError error for filename", filename, e)
                except Exception as ex:
                    print("got error for filename", filename, index)
                    print("Unexpected error:", sys.exc_info()[0])

In [4]:
load_fits_data(positive_folder, label=1)
index = 1
print("NOWWWWWWWWW LABEL=0")
load_fits_data(negative_folder, label=0)
print("positive", np.mean(positive_dips), np.median(positive_dips), np.std(positive_dips))
print("negative", np.mean(negative_dips), np.median(negative_dips))

label, dips 1 8684.0
label, dips 1 798.0
label, dips 1 9490.0
label, dips 1 6472.0
label, dips 1 39.0
label, dips 1 8723.0
label, dips 1 5948.0
label, dips 1 12.0
label, dips 1 900.0
label, dips 1 8140.0
label, dips 1 8607.0
label, dips 1 914.0
label, dips 1 550.0
label, dips 1 626.0
label, dips 1 909.0
label, dips 1 6633.0
label, dips 1 288.0
label, dips 1 685.0
label, dips 1 873.0
label, dips 1 154.0
label, dips 1 293.0
label, dips 1 569.0
label, dips 1 6428.0
label, dips 1 293.0
label, dips 1 90.0
label, dips 1 786.0
label, dips 1 6588.0
label, dips 1 8132.0
label, dips 1 788.0
label, dips 1 467.0
label, dips 1 49.0
label, dips 1 9870.0
label, dips 1 558.0
label, dips 1 348.0
label, dips 1 800.0
label, dips 1 278.0
label, dips 1 801.0
label, dips 1 9097.0
label, dips 1 20.0
label, dips 1 621.0
label, dips 1 600.0
label, dips 1 92.0
label, dips 1 161.0
label, dips 1 587.0
label, dips 1 697.0
label, dips 1 9123.0
label, dips 1 7215.0
label, dips 1 169.0
label, dips 1 8239.0
label, dip